In [1]:
#retrieve and store in a list of url_ending. For example: [egcu.org,libertyfirstcu.com, etc]
#loop through this list to have a consolidated "soup" and get 2 separated files: details & reviews of all companies
#connect to Postgre using Psycopg and store as tables there
#set up cron job & automated scraping for new reviews daily, then append them to the table. 

In [2]:
import requests 
from bs4 import BeautifulSoup 

atm_url = 'https://www.trustpilot.com/categories/atm'

BASE_URL = "https://www.trustpilot.com"

In [3]:
#function for html parser
def get_soup(url):
    response = requests.get(url)
    return BeautifulSoup(response.content, "html.parser")

In [4]:
soup = get_soup(atm_url)

In [5]:
#function to scrap all the URLs of business page

def get_company_urls(soup_response):
    company_urls = []
    for a in soup.select("a[name='business-unit-card']"):
        url_subdirectory = a.attrs.get("href")
        company_urls.append(BASE_URL+url_subdirectory)
    return company_urls

In [6]:
#function to get the link of the next page button and scrap content on next page
def get_next_page_url(soup_response):
    return soup.select("a[name='pagination-button-next']")[0].attrs.get("href")

In [7]:
#scrap the list of company URLs
company_urls = []

while soup:
    company_urls.extend(get_company_urls(soup))
    next_page = get_next_page_url(soup)
    if next_page:
        soup = get_soup(BASE_URL+next_page)
    else:
        soup = None

In [8]:
#remove duplicates in the URL list if any

deduplicated_company_urls = set(company_urls)
deduplicated_company_urls

{'https://www.trustpilot.com/review/acmeatm.cash',
 'https://www.trustpilot.com/review/asicminersrig.com',
 'https://www.trustpilot.com/review/asicminertech.com',
 'https://www.trustpilot.com/review/cashexpressllc.com',
 'https://www.trustpilot.com/review/coinhubatm.com',
 'https://www.trustpilot.com/review/cryptobaseatm.com',
 'https://www.trustpilot.com/review/cryptodispensers.com',
 'https://www.trustpilot.com/review/egcu.org',
 'https://www.trustpilot.com/review/heritagevalleyfcu.org',
 'https://www.trustpilot.com/review/koinkryptatm.com',
 'https://www.trustpilot.com/review/kryptominerstech.com',
 'https://www.trustpilot.com/review/libertyfirstcu.com',
 'https://www.trustpilot.com/review/meriwest.com',
 'https://www.trustpilot.com/review/northone.com',
 'https://www.trustpilot.com/review/pnc.com',
 'https://www.trustpilot.com/review/slide2thrive.com',
 'https://www.trustpilot.com/review/swadesh.co',
 'https://www.trustpilot.com/review/thepaymenthq.com',
 'https://www.trustpilot.co

In [9]:
import pandas as pd

In [15]:
def parse_company_data(sub_soup):
    review_data = []
    name = sub_soup.find('span', attrs={'class': 'typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM'}).text.strip()
    review_stars = sub_soup.find_all('div', attrs={'class': 'star-rating_starRating__4rrcf star-rating_medium__iN6Ty'})
    stars = [stars.find('img')['alt'].replace('Rated ', '').replace(' stars', '') for stars in review_stars]
    review_dates = sub_soup.find('time', attrs={'class': '', 'data-service-review-date-time-ago': 'true'})
    review_title = sub_soup.find('h2', attrs={'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17'})
    reviewer_name = sub_soup.find('span', attrs={'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'})
    review_text = sub_soup.find('p', attrs={'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
    experience_date = sub_soup.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17'})
    review_reply_text = sub_soup.find('p', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX'})
    reply_date = sub_soup.find('time', attrs={'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_replyDate__Iem0_'})
    star = stars[0] if stars else None
    title = review_title.text.strip() if review_title else None
    reviewer = reviewer_name.text.strip() if reviewer_name else None
    text = review_text.text.strip() if review_text else None
    experience = experience_date.text.split(':')[-1].strip() if experience_date else None
    review_date = review_dates.get('datetime').split('T')[0].strip() if review_dates else None
    reply_date = reply_date.text.strip() if reply_date else None
    reply_text = review_reply_text.text.strip() if review_reply_text else None
    review_data.append([name, star, title, reviewer, text, experience, review_date, reply_date, reply_text])
    return review_data

In [16]:
reviews_data = []
for company_url in deduplicated_company_urls:
    company_page = get_soup(company_url)
    reviews_data.append(parse_company_data(company_page))

In [17]:
reviews_data

[[['PNC Bank',
   'TrustScore 1.5 out of 5',
   'PNC bank Monroe NON-EXISTENT CUSTOMER SERVICE',
   'Marcia Wassner Menaker',
   'PNC bank Monroe, NJ is worse than bad!! Minus customer service!! Had to change my account because of fraudulent activity. Got my new card. Customer service unable to activate it, returned it to be to try!! UI was unable to activate it AND NO ONE WOULD HELP!!',
   'July 10, 2023',
   '2023-07-18',
   None,
   None]],
 [['Kryptominerstech',
   'TrustScore 3.5 out of 5',
   'Express delivery as stipulated on the…',
   'Mark George',
   'Express delivery as stipulated on the website is as accurate as it is written. My KD5 was  delivered in perfect condition with the promised Graphic cards. If I hadn’t ordered a few others from another provider I definitely would’ve gotten more from these guys. Now I can continue my minning journey.',
   'December 10, 2021',
   '2021-12-10',
   None,
   None]],
 [['Heritage Valley Federal Credit Union',
   'TrustScore 4 out of 5'

In [13]:
columns = ['company_name','review_star', 'review_title', 'reviewer_name', 'review_text', 'experience_date', 'review_date', 'reply_date', 'review_reply_text']
df_reviews = pd.DataFrame(data=reviews_data, columns=columns)

df_reviews.head(20)

ValueError: 9 columns passed, passed data had 1 columns

In [ ]:
#Establish connection with PostgreSQL using psycopg2

import psycopg2
import numpy as np
import psycopg2.extras as extras

#Function to insert values into existing table
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    col = ','.join(list(df.columns))
    # SQL query to execute
    query = "DELETE FROM %s; INSERT INTO %s(%s) VALUES %%s" % (table, table, col)
    
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()
  
  
conn = psycopg2.connect(
    database="atm_scraping", user='postgres', password='postgres', host='127.0.0.1', port='5432'
)

In [ ]:
execute_values(conn, df_reviews, 'reviews')

the dataframe is inserted
